# Training DRKG Using TransE_L2
This notebook shows how to train DRKG embeddings using TransE_L2

## Install DGL-KE
Before training the model, we need to install dgl and dgl-ke packages as well as other dependencies. 

In [1]:
!pip3 install torch
!pip3 install dgl 
!pip3 install dglke

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Prepare train/valid/test set
Before training, we need to split the original drkg into train/valid/test set with a 9:0.5:0.5 manner.

In [2]:
import pandas as pd
import numpy as np

drkg_file = '~/covid/data/processed/filtered/drkg.tsv'
df = pd.read_csv(drkg_file, sep="\t")
triples = df.values.tolist()

We get 5,869,293 triples, now we will split them into three files

In [3]:
num_triples = len(triples)
num_triples

5869293

In [8]:
# Please make sure the output directory exist.
seed = np.arange(num_triples)
np.random.shuffle(seed)

train_cnt = int(num_triples * 0.9)
valid_cnt = int(num_triples * 0.05)
train_set = seed[:train_cnt]
train_set = train_set.tolist()
valid_set = seed[train_cnt:train_cnt+valid_cnt].tolist()
test_set = seed[train_cnt+valid_cnt:].tolist()

with open("train/drkg_train_04232020.tsv", 'w+') as f:
    for idx in train_set:
        f.writelines("{}\t{}\t{}\n".format(triples[idx][0], triples[idx][1], triples[idx][2]))
        
with open("train/drkg_valid_04232020.tsv", 'w+') as f:
    for idx in valid_set:
        f.writelines("{}\t{}\t{}\n".format(triples[idx][0], triples[idx][1], triples[idx][2]))

with open("train/drkg_test_04232020.tsv", 'w+') as f:
    for idx in test_set:
        f.writelines("{}\t{}\t{}\n".format(triples[idx][0], triples[idx][1], triples[idx][2]))

## Training TransE_l2 model
We can training the TransE_l2 model by simplying using DGL-KE command line. For more information about using DGL-KE please refer to https://github.com/awslabs/dgl-ke.

Here we train the model using 8 GPUs on an AWS p3.16xlarge instance.

In [10]:
!DGLBACKEND=pytorch dglke_train --dataset DRKG --data_path ./train --data_files drkg_train_04232020.tsv drkg_valid_04232020.tsv drkg_test_04232020.tsv --format 'raw_udd_hrt' --model_name TransE_l2 --batch_size 1024 \
--neg_sample_size 256 --hidden_dim 1000 --gamma 12.0 --lr 0.1 --max_step 100000 --log_interval 1000 --batch_size_eval 16 -adv --regularization_coef 1.00E-07 --test --num_thread 1 --gpu 0 1 2 3 4 5 6 7 --num_proc 8 --neg_sample_size_eval 10000 --async_update

Logs are being recorded at: ckpts/TransE_l2_DRKG_0/train.log
Reading train triples....
Finished. Read 5282363 train triples.
Reading valid triples....
Finished. Read 293464 valid triples.
Reading test triples....
Finished. Read 293466 test triples.
|Train|: 5282363
random partition 5282363 edges into 8 parts
part 0 has 660296 edges
part 1 has 660296 edges
part 2 has 660296 edges
part 3 has 660296 edges
part 4 has 660296 edges
part 5 has 660296 edges
part 6 has 660296 edges
part 7 has 660291 edges
/home/ubuntu/.local/lib/python3.6/site-packages/dgl/base.py:25: UserWarning: multigraph will be deprecated.DGL will treat all graphs as multigraph in the future.
  warnings.warn(msg, warn_type)
|valid|: 293464
|test|: 293466
Total initialize time 44.782 seconds
[0][Train](1000/100000) average pos_loss: 0.5430934987068177
[5][Train](1000/100000) average pos_loss: 0.5210402957201004
[3][Train](1000/100000) average pos_loss: 0.5701334822773934
[4][Train](1000/100000) average pos_loss: 0.567467962

[6][Train](4000/100000) average pos_loss: 0.38574772742390634
[4][Train](4000/100000) average pos_loss: 0.38556837069988253
[7][Train](4000/100000) average pos_loss: 0.3852798019051552
[0][Train](4000/100000) average pos_loss: 0.3861467025578022
[6][Train](4000/100000) average neg_loss: 0.5009094079732895
[2][Train](4000/100000) average pos_loss: 0.3842713999450207
[1][Train](4000/100000) average pos_loss: 0.38598944997787477
[4][Train](4000/100000) average neg_loss: 0.5007612273693085
[5][Train](4000/100000) average pos_loss: 0.38530784979462623
[3][Train](4000/100000) average pos_loss: 0.3856995547413826
[0][Train](4000/100000) average neg_loss: 0.5006060391068459
[7][Train](4000/100000) average neg_loss: 0.5026193494200707
[6][Train](4000/100000) average loss: 0.44332856792211534
[2][Train](4000/100000) average neg_loss: 0.5015288275182247
[5][Train](4000/100000) average neg_loss: 0.5011688435971737
[4][Train](4000/100000) average loss: 0.4431647987663746
[1][Train](4000/100000) ave

[0][Train](7000/100000) average pos_loss: 0.3740928495824337
[2][Train](7000/100000) average pos_loss: 0.3720228536427021
[5][Train](7000/100000) average pos_loss: 0.37317542296648026
[4][Train](7000/100000) average pos_loss: 0.37359496679902077
[7][Train](7000/100000) average pos_loss: 0.3734516824781895
[3][Train](7000/100000) average pos_loss: 0.3739628524184227
[0][Train](7000/100000) average neg_loss: 0.48530394262075427
[6][Train](7000/100000) average pos_loss: 0.3739751055240631
[1][Train](7000/100000) average pos_loss: 0.3738661370873451
[2][Train](7000/100000) average neg_loss: 0.48379684129357337
[4][Train](7000/100000) average neg_loss: 0.48573618361353876
[0][Train](7000/100000) average loss: 0.42969839611649513
[7][Train](7000/100000) average neg_loss: 0.4844230545759201
[5][Train](7000/100000) average neg_loss: 0.48387280040979386
[2][Train](7000/100000) average loss: 0.42790984705090523
[3][Train](7000/100000) average neg_loss: 0.485946365237236
[4][Train](7000/100000) a

[6][Train](10000/100000) average pos_loss: 0.3680349720418453
[1][Train](10000/100000) average pos_loss: 0.3682786843776703
[4][Train](10000/100000) average pos_loss: 0.3678765440285206
[3][Train](10000/100000) average pos_loss: 0.3678952354192734
[6][Train](10000/100000) average neg_loss: 0.4763507960438728
[7][Train](10000/100000) average pos_loss: 0.36765422570705414
[0][Train](10000/100000) average pos_loss: 0.36806571736931804
[2][Train](10000/100000) average pos_loss: 0.36650729525089265
[1][Train](10000/100000) average neg_loss: 0.4757333449423313
[4][Train](10000/100000) average neg_loss: 0.4770167616605759
[5][Train](10000/100000) average pos_loss: 0.3678247874081135
[6][Train](10000/100000) average loss: 0.4221928842961788
[3][Train](10000/100000) average neg_loss: 0.4760484027862549
[0][Train](10000/100000) average neg_loss: 0.47772540724277496
[1][Train](10000/100000) average loss: 0.4220060145556927
[7][Train](10000/100000) average neg_loss: 0.47504470685124395
[4][Train](

[4][Train](13000/100000) average pos_loss: 0.3648424467742443
[7][Train](13000/100000) average pos_loss: 0.36451595157384875
[1][Train](13000/100000) average pos_loss: 0.3646547183394432
[0][Train](13000/100000) average pos_loss: 0.36465843546390536
[4][Train](13000/100000) average neg_loss: 0.4732739342749119
[7][Train](13000/100000) average neg_loss: 0.47098896566033366
[6][Train](13000/100000) average pos_loss: 0.36451650720834733
[2][Train](13000/100000) average pos_loss: 0.3633959721028805
[4][Train](13000/100000) average loss: 0.41905819046497345
[1][Train](13000/100000) average neg_loss: 0.47260514107346535
[0][Train](13000/100000) average neg_loss: 0.4692217562496662
[7][Train](13000/100000) average loss: 0.41775245875120165
[5][Train](13000/100000) average pos_loss: 0.3643389576375484
[6][Train](13000/100000) average neg_loss: 0.47068394741415975
[2][Train](13000/100000) average neg_loss: 0.47059311735630033
[3][Train](13000/100000) average pos_loss: 0.3644305101931095
[4][Tra

...

[6][Train](88000/100000) average pos_loss: 0.34810902857780457
[4][Train](88000/100000) average pos_loss: 0.34822621202468873
[7][Train](88000/100000) average pos_loss: 0.34775805789232256
[5][Train](88000/100000) average pos_loss: 0.34810840094089507
[6][Train](88000/100000) average neg_loss: 0.45311662498116495
[1][Train](88000/100000) average pos_loss: 0.3483476513028145
[4][Train](88000/100000) average neg_loss: 0.45193085619807244
[2][Train](88000/100000) average pos_loss: 0.3477075081467628
[0][Train](88000/100000) average pos_loss: 0.3481006774306297
[7][Train](88000/100000) average neg_loss: 0.4541545600891113
[3][Train](88000/100000) average pos_loss: 0.3481039617061615
[5][Train](88000/100000) average neg_loss: 0.4519494433104992
[6][Train](88000/100000) average loss: 0.4006128270626068
[1][Train](88000/100000) average neg_loss: 0.45214907470345495
[4][Train](88000/100000) average loss: 0.40007853400707244
[0][Train](88000/100000) average neg_loss: 0.45156323915719987
[2][Tra

[2][Train](91000/100000) average pos_loss: 0.34740443098545076
[1][Train](91000/100000) average pos_loss: 0.34817614087462423
[6][Train](91000/100000) average pos_loss: 0.34812156999111177
[7][Train](91000/100000) average pos_loss: 0.3478733884692192
[6][Train](91000/100000) average neg_loss: 0.45344593831896784
[0][Train](91000/100000) average pos_loss: 0.34789281532168387
[2][Train](91000/100000) average neg_loss: 0.4520680052936077
[1][Train](91000/100000) average neg_loss: 0.4525198524594307
[3][Train](91000/100000) average pos_loss: 0.34795162856578826
[5][Train](91000/100000) average pos_loss: 0.3479750445187092
[7][Train](91000/100000) average neg_loss: 0.45142833387851716
[4][Train](91000/100000) average pos_loss: 0.34810248377919195
[0][Train](91000/100000) average neg_loss: 0.45241132044792176
[6][Train](91000/100000) average loss: 0.40078375416994094
[1][Train](91000/100000) average loss: 0.4003479969799519
[2][Train](91000/100000) average loss: 0.39973621800541875
[3][Train

[1][Train](94000/100000) average pos_loss: 0.34864687836170194
[6][Train](94000/100000) average pos_loss: 0.34855081531405446
[0][Train](94000/100000) average pos_loss: 0.34852967378497124
[5][Train](94000/100000) average pos_loss: 0.34850376933813093
[4][Train](94000/100000) average pos_loss: 0.34874068799614905
[3][Train](94000/100000) average pos_loss: 0.3485463584959507
[6][Train](94000/100000) average neg_loss: 0.45221130722761155
[1][Train](94000/100000) average neg_loss: 0.4526648158431053
[0][Train](94000/100000) average neg_loss: 0.45266382017731666
[5][Train](94000/100000) average neg_loss: 0.45159149846434593
[4][Train](94000/100000) average neg_loss: 0.4519243330955505
[6][Train](94000/100000) average loss: 0.4003810612857342
[1][Train](94000/100000) average loss: 0.400655846953392
[3][Train](94000/100000) average neg_loss: 0.451849571287632
[5][Train](94000/100000) average loss: 0.40004763385653497
[4][Train](94000/100000) average loss: 0.4003325108587742
[6][Train](94000/

[1][Train](97000/100000) average pos_loss: 0.34855678683519364
[5][Train](97000/100000) average pos_loss: 0.3479960227310657
[6][Train](97000/100000) average pos_loss: 0.34812519466876984
[0][Train](97000/100000) average pos_loss: 0.3478980329632759
[1][Train](97000/100000) average neg_loss: 0.45211407127976416
[4][Train](97000/100000) average pos_loss: 0.34819215258955954
[7][Train](97000/100000) average pos_loss: 0.3478503919243813
[5][Train](97000/100000) average neg_loss: 0.4526314065158367
[1][Train](97000/100000) average loss: 0.40033542892336843
[3][Train](97000/100000) average pos_loss: 0.34796739387512204
[2][Train](97000/100000) average pos_loss: 0.3475399037897587
[6][Train](97000/100000) average neg_loss: 0.45198479238152506
[0][Train](97000/100000) average neg_loss: 0.45194363078475
[4][Train](97000/100000) average neg_loss: 0.4512644964158535
[5][Train](97000/100000) average loss: 0.400313714325428
[7][Train](97000/100000) average neg_loss: 0.4526033067405224
[1][Train](9

[6][Train](100000/100000) average pos_loss: 0.34782455241680144
[6][Train](100000/100000) average neg_loss: 0.45168747574090956
[1][Train](100000/100000) average pos_loss: 0.34782530000805856
[2][Train](100000/100000) average pos_loss: 0.3474367143213749
[0][Train](100000/100000) average pos_loss: 0.347597069889307
[4][Train](100000/100000) average pos_loss: 0.3480709876418114
[5][Train](100000/100000) average pos_loss: 0.34763763630390165
[7][Train](100000/100000) average pos_loss: 0.34751772952079774
[6][Train](100000/100000) average loss: 0.39975601428747176
[3][Train](100000/100000) average pos_loss: 0.3478694050312042
[4][Train](100000/100000) average neg_loss: 0.45210083028674125
[0][Train](100000/100000) average neg_loss: 0.45095299449563025
[6][Train](100000/100000) average regularization: 0.01713797473348677
[1][Train](100000/100000) average neg_loss: 0.452446064978838
[2][Train](100000/100000) average neg_loss: 0.45263473492860795
[7][Train](100000/100000) average neg_loss: 0

## Get Entity and Relation Embeddings
The resulting model, i.e., the entity and relation embeddings can be found under ./ckpts. (Please refer to the first line of the training log for the specific location.)

The overall process will generate 4 important files:

  - Entity embedding: ./ckpts/<model\_name>_<dataset\_name>_<run_\id>/xxx\_entity.npy
  - Relation embedding: ./ckpts/<model\_name>_<dataset\_name>_<run\_id>/xxx\_relation.npy
  - The entity id mapping, formated in <entity\_name> <entity\_id> pair: <data\_path>/entities.tsv
  - The relation id mapping, formated in <relation\_name> <relation\_id> pair: <data\_path>/relations.tsv

In [12]:
!ls ./ckpts/TransE_l2_DRKG_0/
!ls ./train/

DRKG_TransE_l2_entity.npy  DRKG_TransE_l2_relation.npy	config.json  train.log
drkg_test_04232020.tsv	 drkg_valid_04232020.tsv  relations.tsv
drkg_train_04232020.tsv  entities.tsv


## A Glance of the Entity and Relation Embeddings

In [13]:
node_emb = np.load('./ckpts/TransE_l2_DRKG_0/DRKG_TransE_l2_entity.npy')
relation_emb = np.load('./ckpts/TransE_l2_DRKG_0/DRKG_TransE_l2_relation.npy')

print(node_emb.shape)
print(relation_emb.shape)

(97055, 1000)
(106, 1000)


In [14]:
!DGLBACKEND=pytorch dglke_train --dataset DRKG --data_path ./train --data_files drkg_train_04232020.tsv drkg_valid_04232020.tsv drkg_test_04232020.tsv --format 'raw_udd_hrt' --model_name TransE_l2 --batch_size 1024 \
--neg_sample_size 256 --hidden_dim 400 --gamma 8.0 --lr 0.1 --max_step 100000 --log_interval 1000 --batch_size_eval 16 -adv --regularization_coef 1.00E-07 --test --num_thread 1 --gpu 0 1 2 3 4 5 6 7 --num_proc 8 --neg_sample_size_eval 10000 --async_update

Logs are being recorded at: ckpts/TransE_l2_DRKG_1/train.log
Reading train triples....
Finished. Read 5282363 train triples.
Reading valid triples....
Finished. Read 293464 valid triples.
Reading test triples....
Finished. Read 293466 test triples.
|Train|: 5282363
random partition 5282363 edges into 8 parts
part 0 has 660296 edges
part 1 has 660296 edges
part 2 has 660296 edges
part 3 has 660296 edges
part 4 has 660296 edges
part 5 has 660296 edges
part 6 has 660296 edges
part 7 has 660291 edges
/home/ubuntu/.local/lib/python3.6/site-packages/dgl/base.py:25: UserWarning: multigraph will be deprecated.DGL will treat all graphs as multigraph in the future.
  warnings.warn(msg, warn_type)
|valid|: 293464
|test|: 293466
Total initialize time 20.615 seconds
[0][Train](1000/100000) average pos_loss: 0.518443004477711
[4][Train](1000/100000) average pos_loss: 0.4735140958726406
[2][Train](1000/100000) average pos_loss: 0.4866923135519028
[5][Train](1000/100000) average pos_loss: 0.4840950967

[5][Train](4000/100000) average pos_loss: 0.39114439734816553
[3][Train](4000/100000) average pos_loss: 0.39241599130630495
[7][Train](4000/100000) average pos_loss: 0.3917536036968231
[2][Train](4000/100000) average pos_loss: 0.3912761518359184
[0][Train](4000/100000) average pos_loss: 0.3901140390336513
[1][Train](4000/100000) average pos_loss: 0.39163482999801635
[5][Train](4000/100000) average neg_loss: 0.4842955028414726
[4][Train](4000/100000) average pos_loss: 0.39188837897777556
[6][Train](4000/100000) average pos_loss: 0.39118785148859025
[3][Train](4000/100000) average neg_loss: 0.4853986560702324
[7][Train](4000/100000) average neg_loss: 0.48373359072208405
[2][Train](4000/100000) average neg_loss: 0.48399649569392206
[0][Train](4000/100000) average neg_loss: 0.4852101063132286
[5][Train](4000/100000) average loss: 0.4377199502289295
[7][Train](4000/100000) average loss: 0.43774359723925593
[1][Train](4000/100000) average neg_loss: 0.484716149777174
[2][Train](4000/100000) a

[7][Train](7000/100000) average pos_loss: 0.38538453468680384
[6][Train](7000/100000) average pos_loss: 0.3850784168541431
[5][Train](7000/100000) average pos_loss: 0.3844693741500378
[7][Train](7000/100000) average neg_loss: 0.47223537027835844
[1][Train](7000/100000) average pos_loss: 0.3854949359893799
[6][Train](7000/100000) average neg_loss: 0.4719335885643959
[4][Train](7000/100000) average pos_loss: 0.38573417681455613
[2][Train](7000/100000) average pos_loss: 0.38533262223005293
[3][Train](7000/100000) average pos_loss: 0.38584856188297273
[0][Train](7000/100000) average pos_loss: 0.38402094262838365
[1][Train](7000/100000) average neg_loss: 0.47248606371879576
[5][Train](7000/100000) average neg_loss: 0.47201943787932393
[7][Train](7000/100000) average loss: 0.42880995267629624
[6][Train](7000/100000) average loss: 0.4285060026049614
[4][Train](7000/100000) average neg_loss: 0.4734678462445736
[2][Train](7000/100000) average neg_loss: 0.47078337413072585
[1][Train](7000/100000

[4][Train](10000/100000) average pos_loss: 0.382352078974247
[7][Train](10000/100000) average pos_loss: 0.382113403737545
[4][Train](10000/100000) average neg_loss: 0.46654983088374136
[5][Train](10000/100000) average pos_loss: 0.38115957939624784
[6][Train](10000/100000) average pos_loss: 0.38191007113456726
[2][Train](10000/100000) average pos_loss: 0.3820560578405857
[0][Train](10000/100000) average pos_loss: 0.3810635064840317
[4][Train](10000/100000) average loss: 0.424450955092907
[1][Train](10000/100000) average pos_loss: 0.3821255152821541
[3][Train](10000/100000) average pos_loss: 0.38249853858351707
[7][Train](10000/100000) average neg_loss: 0.4653695250451565
[5][Train](10000/100000) average neg_loss: 0.46610592725872996
[6][Train](10000/100000) average neg_loss: 0.4664390080869198
[4][Train](10000/100000) average regularization: 0.0126785248555243
[0][Train](10000/100000) average neg_loss: 0.46651962539553643
[1][Train](10000/100000) average neg_loss: 0.4674096710383892
[3]

[1][Train](13000/100000) average pos_loss: 0.38049122726917267
[6][Train](13000/100000) average pos_loss: 0.38019253075122833
[4][Train](13000/100000) average pos_loss: 0.38086729100346567
[1][Train](13000/100000) average neg_loss: 0.4617513554394245
[6][Train](13000/100000) average neg_loss: 0.4636679612100124
[3][Train](13000/100000) average pos_loss: 0.38092730948328973
[4][Train](13000/100000) average neg_loss: 0.46315002435445785
[2][Train](13000/100000) average pos_loss: 0.38051011103391646
[0][Train](13000/100000) average pos_loss: 0.37945959565043447
[1][Train](13000/100000) average loss: 0.4211212915480137
[7][Train](13000/100000) average pos_loss: 0.38035721361637115
[6][Train](13000/100000) average loss: 0.42193024611473084
[5][Train](13000/100000) average pos_loss: 0.37986607071757317
[3][Train](13000/100000) average neg_loss: 0.4627438068091869
[4][Train](13000/100000) average loss: 0.42200865763425827
[2][Train](13000/100000) average neg_loss: 0.4647244846820831
[1][Train

...

[7][Train](88000/100000) average pos_loss: 0.37112678691744805
[3][Train](88000/100000) average pos_loss: 0.3713449018001556
[7][Train](88000/100000) average neg_loss: 0.45137333884835246
[3][Train](88000/100000) average neg_loss: 0.4511837086081505
[4][Train](88000/100000) average pos_loss: 0.3712803406715393
[1][Train](88000/100000) average pos_loss: 0.3710347553193569
[7][Train](88000/100000) average loss: 0.41125006264448166
[3][Train](88000/100000) average loss: 0.4112643046379089
[2][Train](88000/100000) average pos_loss: 0.37138148602843285
[0][Train](88000/100000) average pos_loss: 0.37119038599729537
[6][Train](88000/100000) average pos_loss: 0.37083083954453466
[7][Train](88000/100000) average regularization: 0.011890002261847258
[3][Train](88000/100000) average regularization: 0.01188403511699289
[1][Train](88000/100000) average neg_loss: 0.45054594612121585
[5][Train](88000/100000) average pos_loss: 0.37025834029912946
[4][Train](88000/100000) average neg_loss: 0.4508427138

[0][Train](91000/100000) average pos_loss: 0.37120693546533584
[6][Train](91000/100000) average pos_loss: 0.3703345627486706
[3][Train](91000/100000) average pos_loss: 0.37107013422250745
[2][Train](91000/100000) average pos_loss: 0.3707547979950905
[0][Train](91000/100000) average neg_loss: 0.4506051631271839
[1][Train](91000/100000) average pos_loss: 0.3707566618025303
[6][Train](91000/100000) average neg_loss: 0.4505100650191307
[7][Train](91000/100000) average pos_loss: 0.37079657396674154
[4][Train](91000/100000) average pos_loss: 0.3709294665753841
[3][Train](91000/100000) average neg_loss: 0.451171582877636
[5][Train](91000/100000) average pos_loss: 0.3703346208333969
[2][Train](91000/100000) average neg_loss: 0.44990244537591934
[0][Train](91000/100000) average loss: 0.4109060489833355
[1][Train](91000/100000) average neg_loss: 0.45092372846603396
[6][Train](91000/100000) average loss: 0.4104223138988018
[7][Train](91000/100000) average neg_loss: 0.4515392163991928
[4][Train](9

[5][Train](94000/100000) average pos_loss: 0.3703631346523762
[7][Train](94000/100000) average pos_loss: 0.371000203371048
[5][Train](94000/100000) average neg_loss: 0.4502623769044876
[0][Train](94000/100000) average pos_loss: 0.3707053539454937
[1][Train](94000/100000) average pos_loss: 0.37111860331892965
[2][Train](94000/100000) average pos_loss: 0.3708407393991947
[3][Train](94000/100000) average pos_loss: 0.37141015446186065
[4][Train](94000/100000) average pos_loss: 0.3714257825314999
[5][Train](94000/100000) average loss: 0.41031275591254235
[7][Train](94000/100000) average neg_loss: 0.4503906935155392
[6][Train](94000/100000) average pos_loss: 0.3707499108016491
[0][Train](94000/100000) average neg_loss: 0.45035680174827575
[1][Train](94000/100000) average neg_loss: 0.45110243836045266
[3][Train](94000/100000) average neg_loss: 0.4496183999478817
[2][Train](94000/100000) average neg_loss: 0.45100562435388564
[4][Train](94000/100000) average neg_loss: 0.4517751088440418
[5][Tra

[4][Train](97000/100000) average pos_loss: 0.37106445026397705
[7][Train](97000/100000) average pos_loss: 0.3707465381026268
[1][Train](97000/100000) average pos_loss: 0.3710870732665062
[4][Train](97000/100000) average neg_loss: 0.4506698811948299
[7][Train](97000/100000) average neg_loss: 0.4506976289451122
[1][Train](97000/100000) average neg_loss: 0.44986230561137197
[2][Train](97000/100000) average pos_loss: 0.3710740321278572
[4][Train](97000/100000) average loss: 0.41086716520786287
[5][Train](97000/100000) average pos_loss: 0.370293586820364
[3][Train](97000/100000) average pos_loss: 0.37121126884222033
[1][Train](97000/100000) average loss: 0.41047468975186346
[7][Train](97000/100000) average loss: 0.41072208344936373
[2][Train](97000/100000) average neg_loss: 0.45069890448451044
[4][Train](97000/100000) average regularization: 0.01185132841207087
[6][Train](97000/100000) average pos_loss: 0.37095904114842415
[5][Train](97000/100000) average neg_loss: 0.45084985530376437
[1][T

[3][Train](100000/100000) average pos_loss: 0.37143891662359235
[2][Train](100000/100000) average pos_loss: 0.3707179232537746
[4][Train](100000/100000) average pos_loss: 0.3712920231819153
[3][Train](100000/100000) average neg_loss: 0.45175992584228514
[5][Train](100000/100000) average pos_loss: 0.37066170933842657
[2][Train](100000/100000) average neg_loss: 0.4511698284447193
[1][Train](100000/100000) average pos_loss: 0.37110361087322236
[7][Train](100000/100000) average pos_loss: 0.3710254788100719
[0][Train](100000/100000) average pos_loss: 0.37077299085259435
[4][Train](100000/100000) average neg_loss: 0.45108175498247144
[3][Train](100000/100000) average loss: 0.41159942135214805
[6][Train](100000/100000) average pos_loss: 0.37077901965379717
[2][Train](100000/100000) average loss: 0.4109438761770725
[5][Train](100000/100000) average neg_loss: 0.450784460157156
[1][Train](100000/100000) average neg_loss: 0.4507868123650551
[0][Train](100000/100000) average neg_loss: 0.4517844901